In [1]:
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy as dc
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
# Load the dataset
data = pd.read_csv('Biotech_preprocessed_dataset.csv')
data

,Time,TOD,Total (W),Phase_A (W),Phase_B (W),Phase_C (W),Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Holiday
0,2024-01-04 15:00:00,15:00:00,821.0,207.0,467.0,147.0,False,False,False,False,True,False,False,False
1,2024-01-04 16:00:00,16:00:00,741.0,298.0,329.0,114.0,False,False,False,False,True,False,False,False
2,2024-01-04 17:00:00,17:00:00,776.0,363.0,278.0,135.0,False,False,False,False,True,False,False,False
3,2024-01-04 18:00:00,18:00:00,1260.0,659.0,419.0,182.0,False,False,False,False,True,False,False,False
4,2024-01-04 19:00:00,19:00:00,973.0,381.0,410.0,182.0,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4454,2024-07-08 05:00:00,05:00:00,1040.0,635.0,259.0,146.0,False,True,False,False,False,False,False,False
4455,2024-07-08 06:00:00,06:00:00,1414.0,1023.0,251.0,140.0,False,True,False,False,False,False,False,False
4456,2024-07-08 07:00:00,07:00:00,571.0,225.0,243.0,103.0,False,True,False,False,False,False,False,False
4457,2024-07-08 08:00:00,08:00:00,784.0,411.0,280.0,93.0,False,True,False,False,False,False,False,False


In [3]:
# To keep it simple, we will predict only on the total active power first
data.drop(['Phase_A (W)', 'Phase_B (W)', 'Phase_C (W)'], axis=1, inplace=True)
data

,Time,TOD,Total (W),Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Holiday
0,2024-01-04 15:00:00,15:00:00,821.0,False,False,False,False,True,False,False,False
1,2024-01-04 16:00:00,16:00:00,741.0,False,False,False,False,True,False,False,False
2,2024-01-04 17:00:00,17:00:00,776.0,False,False,False,False,True,False,False,False
3,2024-01-04 18:00:00,18:00:00,1260.0,False,False,False,False,True,False,False,False
4,2024-01-04 19:00:00,19:00:00,973.0,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
4454,2024-07-08 05:00:00,05:00:00,1040.0,False,True,False,False,False,False,False,False
4455,2024-07-08 06:00:00,06:00:00,1414.0,False,True,False,False,False,False,False,False
4456,2024-07-08 07:00:00,07:00:00,571.0,False,True,False,False,False,False,False,False
4457,2024-07-08 08:00:00,08:00:00,784.0,False,True,False,False,False,False,False,False


In [4]:
# Bring the Total (W) column to the last
power_col = data.pop('Total (W)')
data.insert(len(data.columns), power_col.name, power_col)
data

,Time,TOD,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Holiday,Total (W)
0,2024-01-04 15:00:00,15:00:00,False,False,False,False,True,False,False,False,821.0
1,2024-01-04 16:00:00,16:00:00,False,False,False,False,True,False,False,False,741.0
2,2024-01-04 17:00:00,17:00:00,False,False,False,False,True,False,False,False,776.0
3,2024-01-04 18:00:00,18:00:00,False,False,False,False,True,False,False,False,1260.0
4,2024-01-04 19:00:00,19:00:00,False,False,False,False,True,False,False,False,973.0
...,...,...,...,...,...,...,...,...,...,...,...
4454,2024-07-08 05:00:00,05:00:00,False,True,False,False,False,False,False,False,1040.0
4455,2024-07-08 06:00:00,06:00:00,False,True,False,False,False,False,False,False,1414.0
4456,2024-07-08 07:00:00,07:00:00,False,True,False,False,False,False,False,False,571.0
4457,2024-07-08 08:00:00,08:00:00,False,True,False,False,False,False,False,False,784.0


In [5]:
# extract hour from 'TOD'
data['Time'] = pd.to_datetime(data['Time'])
data['TOD'] = data['Time'].dt.hour

# Since we need only the hour of day, drop the 'Time column'
data.drop('Time', axis=1, inplace=True)
data

,TOD,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Holiday,Total (W)
0,15,False,False,False,False,True,False,False,False,821.0
1,16,False,False,False,False,True,False,False,False,741.0
2,17,False,False,False,False,True,False,False,False,776.0
3,18,False,False,False,False,True,False,False,False,1260.0
4,19,False,False,False,False,True,False,False,False,973.0
...,...,...,...,...,...,...,...,...,...,...
4454,5,False,True,False,False,False,False,False,False,1040.0
4455,6,False,True,False,False,False,False,False,False,1414.0
4456,7,False,True,False,False,False,False,False,False,571.0
4457,8,False,True,False,False,False,False,False,False,784.0


In [ ]:
# set up the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
def prepare_df(df, n_steps: int):
  df = dc(df)
  df.set_index('Time', inplace=True)

  for i in range(1, n_steps+1):
    df[f'Power[t-{i}] (W)'] = df['Total (W)'].shift(i)
  df.dropna(inplace=True)
  return df

lookback = 6
shifted_df = prepare_df(df_total, lookback)
shifted_df


,Total (W),Power[t-1] (W),Power[t-2] (W),Power[t-3] (W),Power[t-4] (W),Power[t-5] (W),Power[t-6] (W)
Time,,,,,,,
2024-01-04 21:00:00,1812,929.0,973.0,1260.0,776.0,741.0,821.0
2024-01-04 22:00:00,715,1812.0,929.0,973.0,1260.0,776.0,741.0
2024-01-04 23:00:00,959,715.0,1812.0,929.0,973.0,1260.0,776.0
2024-01-05 00:00:00,834,959.0,715.0,1812.0,929.0,973.0,1260.0
2024-01-05 01:00:00,694,834.0,959.0,715.0,1812.0,929.0,973.0
...,...,...,...,...,...,...,...
2024-07-08 05:00:00,1040,645.0,1288.0,1200.0,827.0,934.0,1358.0
2024-07-08 06:00:00,1414,1040.0,645.0,1288.0,1200.0,827.0,934.0
2024-07-08 07:00:00,571,1414.0,1040.0,645.0,1288.0,1200.0,827.0


In [ ]:
shifted_df_as_np = shifted_df.to_numpy()
shifted_df_as_np.shape

(4808, 7)

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
shifted_df_as_np = scaler.fit_transform(shifted_df_as_np)
shifted_df_as_np

array([[-0.55595132, -0.8180469 , -0.80498664, ..., -0.86346097,
        -0.87384981, -0.85010389],
       [-0.88156723, -0.55595132, -0.8180469 , ..., -0.71979816,
        -0.86346097, -0.87384981],
       [-0.80914218, -0.88156723, -0.55595132, ..., -0.80498664,
        -0.71979816, -0.86346097],
       ...,
       [-0.92430988, -0.67408727, -0.78509944, ..., -0.71148709,
        -0.7376076 , -0.84832294],
       [-0.86108638, -0.92430988, -0.67408727, ..., -0.90234491,
        -0.71148709, -0.7376076 ],
       [-0.55238943, -0.86108638, -0.92430988, ..., -0.78509944,
        -0.90234491, -0.71148709]])

In [ ]:
# separate features from labels
X = shifted_df_as_np[:, 1:]
y = shifted_df_as_np[:, 0]

X.shape, y.shape

((4808, 6), (4808,))

In [ ]:
# flip the columns so that t-1 is the latest column and t-lookback is the last column
X = dc(np.flip(X, axis=1))
X

array([[-0.85010389, -0.87384981, -0.86346097, -0.71979816, -0.80498664,
        -0.8180469 ],
       [-0.87384981, -0.86346097, -0.71979816, -0.80498664, -0.8180469 ,
        -0.55595132],
       [-0.86346097, -0.71979816, -0.80498664, -0.8180469 , -0.55595132,
        -0.88156723],
       ...,
       [-0.84832294, -0.7376076 , -0.71148709, -0.90234491, -0.78509944,
        -0.67408727],
       [-0.7376076 , -0.71148709, -0.90234491, -0.78509944, -0.67408727,
        -0.92430988],
       [-0.71148709, -0.90234491, -0.78509944, -0.67408727, -0.92430988,
        -0.86108638]])

In [ ]:
# set the split index to separate training data and test data
split_index = int(len(X) * 0.8)
split_index


3846

In [ ]:
# separate the training and test datasets
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3846, 6), (962, 6), (3846,), (962,))

In [ ]:
# convert numpy into tensors to work with pytorch
X_train = torch.tensor(X_train).float()
X_test = torch.tensor(X_test).float()
y_train = torch.tensor(y_train).float()
y_test = torch.tensor(y_test).float()

X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([3846, 6]),
 torch.Size([962, 6]),
 torch.Size([3846]),
 torch.Size([962]))

In [ ]:
# torch dataset
class TimeSeriesDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return len(self.X)

  def __getitem__(self, i):
    return self.X[i], self.y[i]

train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)

In [ ]:
# torch dataloader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for _, batch in enumerate(train_loader):
  x_batch, y_batch = batch[0].to(device), batch[1].to(device)
  print(x_batch.shape, y_batch.shape)
  break


torch.Size([32, 6]) torch.Size([32])


In [ ]:
# LSTM model
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_stacked_layers):
    super().__init__()
    self.hidden_size = hidden_size
    self.num_stacked_layers = num_stacked_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, 1)

  def forward(self, x):
    batch_size = x.size(0)
    h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
    c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)

    out, _ = self.lstm(x, (h0, c0))
    out = self.fc(out[:, -1, :])
    return out

model = LSTM(1,4,1)
model.to(device)
model



LSTM(
  (lstm): LSTM(1, 4, batch_first=True)
  (fc): Linear(in_features=4, out_features=1, bias=True)
)

In [ ]:
def train_one_epoch():
  model.train(True)
  print(f"Epoch: {epoch+1}")
  running_loss = 0.0
  for batch_index, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)

    output = model(x_batch)
    loss = loss_function(output, y_batch)
    running_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch_index % 100 == 99:
      avg_loss_across_batches = running_loss/100
      print(f"Batch {batch_index+1} loss: {avg_loss_across_batches}")
      running_loss = 0.0

    print()


In [ ]:
def validate_one_epoch():
    model.train(False)
    running_loss = 0.0

    for batch_index, batch in enumerate(test_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        with torch.no_grad():
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(test_loader)

    print('Val Loss: {0:.3f}'.format(avg_loss_across_batches))
    print('***************************************************')
    print()

In [ ]:
learning_rate = 0.001
num_epochs = 10
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train_one_epoch()
    validate_one_epoch()

Epoch: 1


RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors